# ABM Design 

The notebook contains both testing code as well as thoughts regarding how agents/households are designed within the simulation

# Configuration JSON

We have the simulation configurations within the configuration folder inside a .json file 

In [1]:
import json 

with open("configurations/single_run.json", 'r') as config_file:
    config_dict = json.load(config_file)

## Model

### Model 

The model needs to have the following variables, the values discussed come from the Monroe et. al simulation model code. 


In [2]:
class Market:
    def __init__(self, config):
        # Global Variables for accurately calculating solar production from fixed tilt arrays
        self.latitude = 20.77 
        self.longitude = 156.92
        self.local_time_meridian = 150.0


        self.sim_config = config # simulation configuration for one run. 
        #households 
        self.households  = [] #probably going to hold everything 

        # Area of benchmark house which represents residential hourly demand
        self.benchmark_area = 2023.0

        # Retail Electricity Rate --> Also used as the penalty rate
        self.retail_rate = 0.351
        # Avoided Fuel Cost Rate
        self.avoided_fuel_cost_rate = 0.0932

        # Date and Time Configuration
        self.days_in_month = 31  # Example: March has 31 days
        self.minutes_in_month = self.days_in_month * 24 * 60
        self.hours_in_month = self.days_in_month * 24
        self.day_light_savings = False
        self.begin_month_day = 244

        # Internal State Variables
        self.initialized = False
        self.minute = 0
        self.increment = 60
        self.current_increment = 1
        self.number_increments = self.minutes_in_month // self.increment
        
        # Simulation Configuration
        self.storage_sim = 1
        self.forecasting_method = 0
        self.lag_increments = 1
        self.number_houses = 25 #Number of household agents within the simulation
        self.simulation_steps = 20 
        
        # Arrays and Matrices for Simulation Data
        self.annual_elec_savings = [0.0] * self.simulation_steps
        self.cumulative_elec_savings = [0.0] * self.simulation_steps
        self.total_adopters = [0] * self.simulation_steps
        self.cumulative_peak_capacity = [0.0] * self.simulation_steps
        self.seller_history = [[[0.0, 0.0, 0.0] for _ in range(self.number_increments)] for _ in range(self.number_houses)]
        self.buyer_history = [[[0.0, 0.0] for _ in range(self.number_increments)] for _ in range(self.number_houses)]
        self.overproduction = [[0.0] * self.number_increments for _ in range(self.number_houses)]
        self.shortfall = [[0.0] * self.number_increments for _ in range(self.number_houses)]
        self.overbought = [[0.0] * self.number_increments for _ in range(self.number_houses)]
        self.underbought = [[0.0] * self.number_increments for _ in range(self.number_houses)]
        self.daytime_demand = [[0.0] * self.number_increments for _ in range(self.number_houses)]
        self.excess_solar = [[0.0] * self.number_increments for _ in range(self.number_houses)]
        self.interval_average_seller_price = [0.0] * self.number_increments
        self.running_average_seller_price = [0.0] * self.number_increments

    def print_initialization(self): 

        print(f"Latitude: {self.latitude}")
        print(f"Longitude: {self.longitude}")
        print(f"Local Time Meridian: {self.local_time_meridian}")
        print(f"Benchmark Area: {self.benchmark_area}")
        print(f"Retail Rate: {self.retail_rate}")
        print(f"Avoided Fuel Cost Rate: {self.avoided_fuel_cost_rate}")
        print(f"Days in Month: {self.days_in_month}")
        print(f"Minutes in Month: {self.minutes_in_month}")
        print(f"Hours in Month: {self.hours_in_month}")
        print(f"Day Light Savings: {self.day_light_savings}")
        print(f"Begin Month Day: {self.begin_month_day}")
        print(f"Initialized: {self.initialized}")
        print(f"Minute: {self.minute}")
        print(f"Increment: {self.increment}")
        print(f"Current Increment: {self.current_increment}")
        print(f"Number of Increments: {self.number_increments}")
        print(f"Storage Simulation Option: {self.storage_sim}")
        print(f"Forecasting Method: {self.forecasting_method}")
        print(f"Lag Increments: {self.lag_increments}")
        print(f"Number of Houses: {self.number_houses}")
        print(f"Simulation Steps: {self.simulation_steps}")

        print(f"Annual Electricity Savings: {self.annual_elec_savings}")
        print(f"Cumulative Electricity Savings: {self.cumulative_elec_savings}")
        print(f"Total Adopters: {self.total_adopters}")
        print(f"Cumulative Peak Capacity: {self.cumulative_peak_capacity}")
        print(f"Interval Average Seller Price: {self.interval_average_seller_price}")
        print(f"Running Average Seller Price: {self.running_average_seller_price}")

        print(f"seller_history = {self.seller_history}")
        print(f"buyer_history = {self.buyer_history}")
        print(f"shortfall = {self.shortfall}")
        print(f"overproduction = {self.overproduction}")


    def initialize_households(self):
        """ 
        Initialize the different households according to their 
        
        INPUT: 
            config (dict):contains the relevant information that 
                         houses might need to initialize their relevant 
                         states. 
        
        RETURN: 
            None
        """
        household_configs = self.sim_config["household_params"]
        for hh_config in household_configs: 
            pass


            
    def run_simulation(self): 
        """  
        #TODO: update as you go:
        Initializes households prosumers and consumers and then runs the entire 
        simulation for self.number_increments timesteps.

        INPUT: 
        
        RETURN: 
            NONE
        
        """
        print(f"-"*50, "RUN SIMULATION", f"-"*50)
        print(f"")

        self.initialize_households()

        for timestep in range(self.number_increments): 
            print(f"-"*20, f"timestep : {timestep}", f"-"*20)

        





### Households

In [3]:
from scipy.stats import norm
import numpy as np

class Household:
    def __init__(self, index, has_pv, floor_area):
        self.index = index
        self.has_pv = has_pv
        self.floor_area = floor_area
        self.roof_area = self.floor_area * 1.12
        self.days_month = 31
        self.minutes_month = self.days_month * 24 * 60
        self.increment = 60
        self.number_increments = self.minutes_month // self.increment
        self.increments_per_hour = 60 // self.increment
        self.increments_per_day = 1440 // self.increment
        self.start_charge = 9
        self.stop_charge = 14
        self.start_charge_inc = self.increments_per_hour * self.start_charge
        self.stop_charge_inc = self.increments_per_hour * self.stop_charge
        self.start_discharge = 17
        self.stop_discharge = 22
        self.start_discharge_inc = self.increments_per_hour * self.start_discharge
        self.stop_discharge_inc = self.increments_per_hour * self.stop_discharge
        self.cell_efficiency = 0.253
        self.irradiation = 0.0929
        self.solar_proportion = 0.10
        self.price_kw = 3090
        self.interest_rate = 0.07
        self.minimum_charge = 4.05
        self.maximum_charge = 13.5
        self.electricity_use = np.zeros(self.minutes_month)
        self.solar_prod = np.zeros(self.minutes_month)
        self.solar_prod_forecast = np.zeros(self.minutes_month)
        self.solar_prod_forecast_increment = np.zeros(self.number_increments)
        self.demand = np.zeros(self.number_increments)
        self.demand_after_storage_discharge = np.zeros(self.number_increments)
        self.demand_after_solar_storage = np.zeros(self.number_increments)
        self.production = np.zeros(self.number_increments)
        self.production_for_demand = np.zeros(self.number_increments)
        self.production_after_demand = np.zeros(self.number_increments)
        self.production_for_market = np.zeros(self.number_increments)
        self.production_remaining_after_market = np.zeros(self.number_increments)
        self.production_for_storage = np.zeros(self.number_increments)
        self.production_for_utility = np.zeros(self.number_increments)
        self.storage = np.zeros(self.number_increments)
        self.storage_initially_discharged = np.zeros(self.number_increments)
        self.storage_after_initial_discharge = np.zeros(self.number_increments)
        self.storage_discharge_market = np.zeros(self.number_increments)
        self.initialize_elec_attributes() #TODO: remove later, if this doesn't work 

    def initialize_elec_attributes(self):
        self.calc_peak_power()
        self.calc_investment_cost()
        self.calc_annual_prod()
        self.calc_prod_20()

    def calc_peak_power(self):
        self.peak_power = self.irradiation * self.solar_proportion * self.roof_area * self.cell_efficiency

    def calc_investment_cost(self):
        self.investment_costs = self.peak_power * self.price_kw

    def calc_annual_prod(self):
        self.expected_production = self.peak_power * 8760 * 0.23

    def calc_prod_20(self):
        self.expected_production_20 = self.expected_production * 20

    def set_electricity_use(self, elec_use):
        self.electricity_use = elec_use

    def get_electricity_use(self, minute):
        return self.electricity_use[minute]

    def set_solar_production(self, production):
        self.solar_prod = production

    def get_solar_production(self, minute):
        return self.solar_prod[minute]

    def set_forecasted_solar_production(self):
        normal_dist = norm(0, 0.30)
        forecast_multiplier = normal_dist.rvs(size=self.minutes_month) + 1
        self.solar_prod_forecast = self.solar_prod * forecast_multiplier

    def fill_solar_prod_forecast_increment(self):
        for i in range(self.number_increments):
            start_minute = i * self.increment
            end_minute = start_minute + self.increment
            if self.has_pv:
                self.solar_prod_forecast_increment[i] = np.sum(self.solar_prod_forecast[start_minute:end_minute]) / 1000 / 60
            else:
                self.solar_prod_forecast_increment[i] = 0

# Example of using the Household class
household = Household(index=1, has_pv=True, floor_area=1500)
household.set_electricity_use(np.random.rand(household.minutes_month))
household.set_solar_production(np.random.rand(household.minutes_month) * 5)
household.set_forecasted_solar_production()
print(f"Household Demand at Minute 50: {household.get_electricity_use(50)} kWh")
print(f"Household Solar Production at Minute 50: {household.get_solar_production(50)} kWh")


Household Demand at Minute 50: 0.9565810788209996 kWh
Household Solar Production at Minute 50: 3.6859706275329236 kWh


In [ ]:
market = Market(config_dict)

In [ ]:
market.run_simulation()

In [21]:
2188//60 


36

In [20]:
35*60

2100

In [4]:
from abm.model import Market
import json 
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

In [35]:
import json
import plotly.graph_objs as go


def plot_hh_elec_demand(abm_model):
    fig = go.Figure()
    
    # Ensure abm_model.hours_in_month is an array-like object
    x_axis = list(range(len(abm_model.solar_energy_production_arr)))

    for index, hh_obj in enumerate(abm_model.households):
        fig.add_trace(go.Scatter(x=x_axis, y=hh_obj.electricity_use, name=f"{index}"))
    
    # Add title and labels
    fig.update_layout(
        title='Electricity Demand: Always consume Morty never return',
        xaxis_title='time',
        yaxis_title='kw/h2'
    )
    
    fig.show()


def plot_hh_elec_prod(abm_model):

    fig = go.Figure()

    x_axis = list(range(len(abm_model.solar_energy_production_arr)))
    fig.add_trace(go.Scatter(x= list(range(len(abm_model.solar_energy_production_arr))), y = abm_model.solar_energy_production_arr, name= "Solar Irradience (Sun that shines)" ))

    for index, hh_obj in enumerate(abm_model.households):
        fig.add_trace(go.Scatter(x=x_axis, y=hh_obj.solar_prod, name=f"{index}"))
    
    fig.update_layout(
        title='Sun that shines vs Sun we capture',
        xaxis_title='time',
        yaxis_title='kw/h2'
    )

    # Show the plot
    fig.show()

if __name__ == "__main__":
    config_dict = {} 
    with open("configurations/single_run.json", 'r') as config_file:
        config_dict = json.load(config_file)
    
    abm_model = Market(config_dict)
    abm_model.run_simulation()
    
    # plot_hh_elec_demand(abm_model)
    # plot_hh_elec_prod(abm_model)


-------------------------------------------------- RUN SIMULATION --------------------------------------------------
------------------------------ Initializing ------------------------------
------------------------------ Initialization Complete ------------------------------
----- Household: 0 -----
buyer History: 0.0
----- Household: 1 -----
buyer History: 0.0
----- Household: 2 -----
buyer History: 0.0
----- Household: 3 -----
buyer History: 0.0
----- Household: 4 -----
buyer History: 0.0
----- Household: 5 -----
buyer History: 0.0
----- Household: 6 -----
buyer History: 0.0
----- Household: 7 -----
buyer History: 0.0
----- Household: 8 -----
buyer History: 0.0
----- Household: 9 -----
buyer History: 0.0
----- Household: 10 -----
buyer History: 0.0
----- Household: 11 -----
buyer History: 0.0
----- Household: 12 -----
buyer History: 0.0
----- Household: 13 -----
buyer History: 0.0
----- Household: 14 -----
buyer History: 0.0
----- Household: 15 -----
buyer History: 0.0
----- Househ

In [9]:


def plot_hh_elec_demand(abm_model):
    fig = go.Figure()
    x_axis = list(range(len(abm_model.solar_energy_production_arr)))

    for index, hh_obj in enumerate(abm_model.households):
        fig.add_trace(go.Scatter(x= abm_model.hours_in_month, y = hh_obj.electricity_use , name= f"{index}"))
        # Add title and labels
    fig.update_layout(
        title='Electricity Demand: Always consume Morty never return',
        xaxis_title='time',
        yaxis_title='kw/h2'
    )

    # Show the plot
    fig.show()

def plot_hh_elec_prod(abm_model):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x= list(range(len(b))), y = b, name= "solar_irradiance" ))
    fig.add_trace(go.Scatter(x= list(range(len(a))), y = a , name= "actual_production"))
    # Add title and labels
    fig.update_layout(
        title='Sun that shines vs Sun we capture',
        xaxis_title='time',
        yaxis_title='kw/h2'
    )

    # Show the plot
    fig.show()




In [ ]:


fig = go.Figure()

fig.add_trace(go.Scatter(x= list(range(len(b))), y = b, name= "solar_irradiance" ))
fig.add_trace(go.Scatter(x= list(range(len(a))), y = a , name= "actual_production"))
# Add title and labels
fig.update_layout(
    title='Sun the shines vs Sun we capture',
    xaxis_title='time',
    yaxis_title='kw/h2'
)

# Show the plot
fig.show()



## Conversion of Irradiance data to solar energy 

This method processes an array of raw irradiance data, which represents the solar power available per unit area at each minute throughout a month. The method adjusts this raw irradiance to reflect the actual energy that a fixed tilt solar panel would collect, considering the panel's geographic location and orientation relative to the sun. Here’s a step-by-step breakdown of what happens in the method:

1. **Declination Calculation**: Calculates the solar declination angle for each day, which is the angle between the rays of the sun and the plane of the Earth's equator. This angle varies throughout the year and affects the sun’s position in the sky relative to a fixed location on Earth.

2.  **Solar Noon and Hour Angle** : Computes the solar noon (the time when the sun is highest in the sky) for each day, and the hour angle for every minute. The hour angle indicates the sun's position relative to its highest point (solar noon).

3. **Altitude and Azimuth Angles**: Determines the altitude angle (how high the sun is in the sky) and the solar azimuth angle (the compass direction from which the sunlight is coming) for each minute. These angles are crucial for calculating the amount of sunlight that hits the panel directly.

4. **Incidence Angle** : Calculates the angle of incidence, which is the angle at which solar radiation strikes the panel. This angle affects how effectively the panel can convert irradiance into usable energy.

### Component Radiation Calculations:

1. **Beam Radiation**: The direct sunlight that hits the panel.
2. **Diffuse Radiation**: Sunlight scattered by the atmosphere.
3. **Reflected Radiation**: Sunlight reflected off the ground and other surfaces onto the panel.
4. **Total Collected Radiation**: Summarizes the beam, diffuse, and reflected radiation to estimate the total solar radiation captured by the panel per minute.

### Method 2: solarNoon
This auxiliary method supports the first by determining the exact minute of solar noon for each day. Solar noon time varies slightly each day due to the equation of time and the specific longitudinal correction for the location’s difference from the standard time meridian. It also considers whether daylight saving time is in effect, adjusting the solar noon calculation accordingly.

### Application in Solar Energy Simulation
These methods together enable a detailed simulation of solar energy production for fixed tilt solar panels. By adjusting raw irradiance data based on precise calculations of the sun's angles and the panel's orientation, the simulation can predict how much energy the panel will produce at any given minute throughout the month. This is crucial for optimizing solar panel installations, estimating energy yields, and planning for energy needs in solar-reliant systems.

The calculations are based on solar geometry and require accurate geographic data (latitude, longitude) and a detailed understanding of the panel's orientation (tilt angle, azimuth direction). These simulations help in designing more efficient solar power systems by predicting performance across different times of the day and seasons.

In [ ]:
import math
import numpy as np
import pandas as pd 

class SolarModel:
    def __init__(self, latitude, longitude, localTimeMeridian, daysInMonth, beginMonthDay):
        self.latitude = latitude
        self.longitude = longitude
        self.localTimeMeridian = localTimeMeridian
        self.daysInMonth = daysInMonth
        self.beginMonthDay = beginMonthDay
        self.dayLightSavings = False
    
    def adjustIrradiance(self, irradianceArray, problematic_indices=[]):
        minutesInMonth = self.daysInMonth * 1440
        declination = np.zeros(minutesInMonth)
        hourAngle = np.zeros(minutesInMonth)
        altitudeAngle = np.zeros(minutesInMonth)
        solarAzimuth = np.zeros(minutesInMonth)
        C = np.zeros(minutesInMonth)
        incidenceAngle = np.zeros(minutesInMonth)
        beamRad = np.zeros(minutesInMonth)
        diffuseRad = np.zeros(minutesInMonth)
        reflectedRad = np.zeros(minutesInMonth)
        collectorRad = np.zeros(minutesInMonth)
        
        for i in range(self.daysInMonth):
            currentDeclination = 23.45 * math.sin(math.radians((360/365) * (self.beginMonthDay + i)))
            minuteOfSolarNoon = self.solarNoon(self.beginMonthDay + i)
            currentC = 0.095 + (0.04 * math.sin(math.radians((360/365) * (self.beginMonthDay + i - 100))))

            for j in range(1440):
                currentMinute = (1440 * i) + j
                declination[currentMinute] = currentDeclination
                minutesFromSolarNoon = (minuteOfSolarNoon - j)
                hourAngle[currentMinute] = (minutesFromSolarNoon / 60.0) * 15.0
                altitudeAngle[currentMinute] = math.degrees(math.asin(math.cos(math.radians(self.latitude)) * 
                                            math.cos(math.radians(declination[currentMinute])) * 
                                            math.cos(math.radians(hourAngle[currentMinute])) + 
                                            (math.sin(math.radians(self.latitude)) * 
                                             math.sin(math.radians(declination[currentMinute])))))
                solarAzimuth[currentMinute] = math.degrees(math.asin((math.cos(math.radians(declination[currentMinute])) * 
                                          math.sin(math.radians(hourAngle[currentMinute])))/
                                          (math.cos(math.radians(altitudeAngle[currentMinute])))))
                incidenceAngle[currentMinute] = math.degrees(math.acos(math.cos(math.radians(altitudeAngle[currentMinute])) * 
                                           math.cos(math.radians(solarAzimuth[currentMinute])) * 
                                           math.sin(math.radians(self.latitude)) + 
                                           (math.sin(math.radians(altitudeAngle[currentMinute])) * 
                                            math.cos(math.radians(self.latitude)))))
                C[currentMinute] = currentC
                beamRad[currentMinute] = irradianceArray[currentMinute] * math.cos(math.radians(incidenceAngle[currentMinute]))
                diffuseRad[currentMinute] = irradianceArray[currentMinute] * currentC * ((1 + math.cos(math.radians(self.latitude))) / 2)
                reflectedRad[currentMinute] = irradianceArray[currentMinute] * 0.2 * (currentC + math.sin(math.radians(altitudeAngle[currentMinute]))) * ((1 - math.cos(math.radians(self.latitude))) / 2)
                collectorRad[currentMinute] = beamRad[currentMinute] + diffuseRad[currentMinute] + reflectedRad[currentMinute]

                # Debugging output for problematic indices
                if currentMinute in problematic_indices:
                    print(f"Minute {currentMinute}: Irradiance = {irradianceArray[currentMinute]}, "
                          f"BeamRad = {beamRad[currentMinute]}, DiffuseRad = {diffuseRad[currentMinute]}, "
                          f"ReflectedRad = {reflectedRad[currentMinute]}, CollectorRad = {collectorRad[currentMinute]}")

        return collectorRad

    def solarNoon(self, n):
        # if 69 <= n <= 307:
        #     self.dayLightSavings = True
        # else:
        #     self.dayLightSavings = False

        longitudeCorrection = 4.0 * (self.localTimeMeridian - self.longitude)  # Units of minutes
        Bdegrees = (360.0/364.0) * (n - 81)
        Bradians = math.radians(Bdegrees)
        E = (9.87 * math.sin(2 * Bradians)) - (7.53 * math.cos(Bradians)) - (1.5 * math.sin(Bradians))  # Units of minutes
        minutesFromClockNoon = -longitudeCorrection - E
        minutesFromClockNoonInt = int(round(minutesFromClockNoon))
        
        if self.dayLightSavings:
            return (13 * 60) + minutesFromClockNoonInt
        else:
            return (12 * 60) + minutesFromClockNoonInt


In [ ]:
# Example usage
latitude = 20.77
longitude = 156.92
localTimeMeridian = 150.0
daysInMonth = 30
beginMonthDay = 244

# Problematic indices provided earlier
problematic_indices = [384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401,
                       402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 1825,
                       1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840,
                       1841, 1842, 1843, 1844, 1845, 1846, 1847, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856,
                       1857, 1858, 3290, 3291, 3292, 3293, 3294, 3295, 3296, 3297, 4706, 4707, 4708, 4709, 4710,
                       4711, 4712, 4713, 4714, 4715, 4716, 4717, 4718, 4719, 4720, 4721, 4722, 4723, 4724, 4725,
                       4726, 4727, 4728, 4729, 4730, 4731, 4732, 4733, 4734, 4735, 4736, 4737, 4738, 6146, 6147,
                       6148, 6149, 6150, 6151, 6152, 6153, 6154, 6155, 6156, 6157, 6158, 6159, 6160, 6161, 6162,
                       6163, 6164, 6165, 6166, 6167, 6168, 6169, 6170, 6171, 6172, 6173, 6174, 6175, 6176, 6177,
                       6178, 7605, 7606, 7607, 7608, 7609, 7610, 7611, 7612, 7613, 7614, 7615, 7616, 7617, 9025,
                       9026, 9027, 9028, 9029, 9030, 9031, 9032, 9033, 9034, 9035, 9036, 9037, 9038, 9039, 9040,
                       9041, 9042, 9043, 9044, 9045, 9046, 9047, 9048, 9049, 9050, 9051, 9052, 9053, 9054, 9055,
                       9056, 9057, 10465, 10466, 10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475,
                       10476, 10477, 10478, 10479, 10480, 10481, 10482, 10483, 10484, 10485, 10486, 10487, 10488,
                       10489, 10490, 10491, 10492, 10493, 10494, 10495, 10496, 10497, 11910, 11911, 11912, 11913,
                       11914, 11915, 11916, 11917, 11918, 11919, 11920, 11921, 11922, 11923, 11924, 11925, 11926,
                       11927, 11928, 11929, 11930, 11931, 11932, 11933, 11934, 11935, 13345, 13346, 13347, 13348,
                       13349, 13350, 13351, 13352, 13353, 13354, 13355, 13356, 13357, 13358, 13359, 13360, 13361,
                       13362, 13363, 13364, 13365, 13366, 13367, 13368, 13369, 13370, 13371, 13372, 13373, 13374,
                       13375, 13376, 14785, 14786, 14787, 14788, 14789, 14790, 14791, 14792, 14793, 14794, 14795,
                       14796, 14797, 14798, 14799, 14800, 14801, 14802, 14803, 14804, 14805, 14806, 14807, 14808,
                       14809, 14810, 14811, 14812, 14813, 14814, 14815, 16227, 16228, 16229, 16230, 16231, 16232,
                       16233, 16234, 16235, 16236, 16237, 16238, 16239, 16240, 16241, 16242, 16243, 16244, 16245,
                       16246, 16247, 16248, 16249, 16250, 16251, 16252, 16253, 16254, 16255, 16256, 17666, 17667,
                       17668, 17669, 17670, 17671, 17672, 17673, 17674, 17675, 17676, 17677, 17678, 17679, 17680,
                       17681, 17682, 17683, 17684, 17685, 17686, 17687, 17688, 17689, 17690, 17691, 17692, 17693,
                       17694, 17695, 17696, 19105, 19106, 19107, 19108, 19109, 19110, 19111, 19112, 19113, 19114,
                       19115, 19116, 19117, 19118, 19119, 19120, 19121, 19122, 19123, 19124, 19125, 19126, 19127,
                       19128, 19129, 19130, 19131, 19132, 19133, 19134, 19135, 20545, 20546, 20547, 20548, 20549,
                       20550, 20551, 20552, 20553, 20554, 20555, 20556, 20557, 20558, 20559, 20560, 20561, 20562,
                       20563, 20564, 20565, 20566, 20567, 20568, 20569, 20570, 20571, 20572, 20573, 20574, 20575,
                       23432, 23433, 23434, 23435, 23436, 23437, 23438, 23439, 23440, 23441, 23442, 23443, 23444,
                       23445, 23446, 23447, 23448, 23449, 23450, 23451, 23452, 23453, 23454, 24868, 24869, 24870,
                       24871, 24872, 24873, 24874, 24875, 24876, 24877, 24878, 24879, 24880, 24881, 24882, 24883,
                       24884, 24885, 24886, 24887, 24888, 24889, 24890, 24891, 24892, 24893, 24894, 26311, 26312,
                       26313, 26316, 26317, 26318, 26319, 26320, 26321, 26322, 26323, 26324, 26325, 26326, 26327,
                       26328, 26329, 26330, 26331, 26332, 26333, 26334, 27749, 27756, 27757, 27758, 27759, 27760,
                       27761, 27762, 27763, 27766, 27767, 27768, 27769, 27770, 27771, 27772, 29192, 29197, 29198,
                       29199, 29200, 29201, 29202, 29203, 29204, 29205, 29206, 29207, 29208, 29209, 29210, 29211,
                       29212, 29213, 29214, 32072, 32073, 32074, 32075, 32076, 32077, 32078, 32079, 32080, 32081,
                       32082, 32083, 32084, 32085, 32086, 32087, 32088, 32089, 32090, 32091, 32092, 32093, 33509,
                       33510, 33513, 33514, 33515, 33516, 33517, 33518, 33519, 33520, 33521, 33522, 33523, 33524,
                       33525, 33526, 33527, 33528, 33529, 33530, 33531, 33532, 33533, 34948, 34949, 34950, 34951,
                       34952, 34953, 34954, 34955, 34956, 34957, 34958, 34959, 34960, 34961, 34962, 34963, 34964,
                       34965, 34966, 34967, 34968, 34969, 34970, 34971, 34972, 36388, 36389, 36390, 36391, 36392,
                       36393, 36394, 36395, 36396, 36397, 36398, 36399, 36400, 36401, 36402, 36403, 36404, 36405,
                       36406, 36407, 36408, 36409, 36410, 36411, 36412, 37852, 37853, 42152, 42153, 42154, 42155,
                       42156, 42157, 42158, 42159, 42160, 42161, 42162, 42163, 42164, 42165, 42166, 42167, 42168,
                       42169, 42170, 42171, 42172]


len(problematic_indices)
# # Create instance of SolarModel
solar_model = SolarModel(latitude, longitude, localTimeMeridian, daysInMonth, beginMonthDay)

# # Load your irradiance array
irradiance_array = pd.read_csv("data/production_monthly_minutely.csv")
irradiance_array = irradiance_array.squeeze()


#irradiance_array[17660:17700]
# # Call the method with problematic indices for debugging
solar_energy_production = solar_model.adjustIrradiance(irradiance_array, problematic_indices)


### Declination Angle

The declination angle \( \delta \) varies daily and is given by:

$$
\delta = 23.45 \cdot \sin\left(\frac{360}{365} \cdot (n + d - 81)\right)
$$

where:
- \( n \) is the day of the year,
- \( d \) is the day index within the month.

### Solar Noon

Solar noon time \( t_{\text{noon}} \) and correction factors:

$$
t_{\text{noon}} = 720 + 4 \cdot (\text{LSTM} - \text{long}) + E
$$

$$
E = 9.87 \cdot \sin(2B) - 7.53 \cdot \cos(B) - 1.5 \cdot \sin(B)
$$

$$
B = \frac{360}{364} \cdot (n - 81)
$$

where:
- \(\text{LSTM}\) is the local standard time meridian,
- \(\text{long}\) is the longitude.

### Hour Angle

The hour angle \( H \) for a given minute \( m \) of the day is:

$$
H = \frac{m - t_{\text{noon}}}{4}
$$

### Solar Altitude Angle

The solar altitude angle \( \alpha \) is calculated as:

$$
\alpha = \sin^{-1}\left(\sin(\text{lat}) \cdot \sin(\delta) + \cos(\text{lat}) \cdot \cos(\delta) \cdot \cos(H)\right)
$$

where \(\text{lat}\) is the latitude.

### Solar Azimuth Angle

The solar azimuth angle \( Az \) is:

$$
Az = \sin^{-1}\left(\frac{\sin(H) \cdot \cos(\delta)}{\cos(\alpha)}\right)
$$

### Incidence Angle

The incidence angle \( \theta \) for a fixed tilt panel:

$$
\theta = \cos^{-1}\left(\sin(\text{lat}) \cdot \sin(\alpha) + \cos(\text{lat}) \cdot \cos(\alpha) \cdot \cos(Az)\right)
$$

### Radiation Components

1. **Beam Radiation**:
   $$
   I_{\text{beam}} = I \cdot \cos(\theta)
   $$
2. **Diffuse Radiation**:
   $$
   I_{\text{diffuse}} = I \cdot C \cdot \frac{1 + \cos(\text{lat})}{2}
   $$
3. **Reflected Radiation**:
   $$
   I_{\text{reflected}} = I \cdot 0.2 \cdot (C + \sin(\alpha)) \cdot \frac{1 - \cos(\text{lat})}{2}
   $$

### Total Collector Radiation

The total radiation collected by the collector \( I_{\text{total}} \):

$$
I_{\text{total}} = I_{\text{beam}} + I_{\text{diffuse}} + I_{\text{reflected}}
$$


## Going from maximum irradiance that might fall to how much can your solar panel capture.

At minute j within the given month 
$$
\text{individual\_production\_ledger}[j] = \text{adjusted\_individual\_minute\_production}[j] \times 0.092903 \times (\text{roof\_area} \times 0.10) \times 0.253 \times 0.77
$$


The paper provides context for the factors and assumptions involved in the model code for calculating individual solar production for houses. Here are the details about the constants used in the calculation:

1. **0.092903**: This value is the conversion factor from square feet to square meters. Since the roof area is likely measured in square feet, this conversion factor is used to convert the roof area into square meters, which is the standard unit for irradiance and other related calculations.

2. **0.10**: This is the assumed proportion of the roof area that is available for installing solar panels. It accounts for factors like shading, roof structure, and practical installation constraints. The paper mentions that 10% of the roof area is considered a reasonable estimate for solar panel coverage.

3. **0.253**: This constant likely represents the panel efficiency or a related factor affecting the conversion of solar irradiance into electricity. It could be the efficiency of the solar panels or an adjustment factor to account for the specific performance characteristics of the installed panels. However, the exact origin of this value isn't detailed in the provided excerpts from the paper.

4. **0.77**: This constant represents the derate factor, which accounts for various losses in the system, such as inefficiencies in the inverter, wiring losses, shading, and other system losses. A derate factor of 77% is mentioned as a default value used by PVWatts, a common tool for estimating solar energy production.

In summary, the calculation for individual production for houses considers the roof area, a realistic coverage proportion for solar panels, the efficiency of the panels, and various system losses to provide a more accurate estimate of solar energy production.

For more detailed information, including the specific source of the 0.253 value, you would need to refer to the specific section of the paper that discusses these assumptions or calculations in detail【53:0†Mainmodelpaper.pdf】.

In [ ]:
import pandas as pd 


irradiance_array = pd.read_csv("data/production_monthly_minutely.csv").squeeze()

# Complete and corrected list of problematic indices
problematic_indices = np.array([384, 385, 386, 387, 388, 389, 390, 391, 392,
                                393, 394, 395, 396, 397, 398, 399, 400, 401,
                                402, 403, 404, 405, 406, 407, 408, 409, 410,
                                411, 412, 413, 414, 415, 416, 417, 418, 1825,
                                1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833,
                                1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841,
                                1842, 1843, 1844, 1845, 1846, 1847, 1849, 1850,
                                1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858,
                                3290, 3291, 3292, 3293, 3294, 3295, 3296, 3297,
                                4706, 4707, 4708, 4709, 4710, 4711, 4712, 4713,
                                4714, 4715, 4716, 4717, 4718, 4719, 4720, 4721,
                                4722, 4723, 4724, 4725, 4726, 4727, 4728, 4729,
                                4730, 4731, 4732, 4733, 4734, 4735, 4736, 4737,
                                4738, 6146, 6147, 6148, 6149, 6150, 6151, 6152,
                                6153, 6154, 6155, 6156, 6157, 6158, 6159, 6160,
                                6161, 6162, 6163, 6164, 6165, 6166, 6167, 6168,
                                6169, 6170, 6171, 6172, 6173, 6174, 6175, 6176,
                                6177, 6178, 7605, 7606, 7607, 7608, 7609, 7610,
                                7611, 7612, 7613, 7614, 7615, 7616, 7617, 9025,
                                9026, 9027, 9028, 9029, 9030, 9031, 9032, 9033,
                                9034, 9035, 9036, 9037, 9038, 9039, 9040, 9041,
                                9042, 9043, 9044, 9045, 9046, 9047, 9048, 9049,
                                9050, 9051, 9052, 9053, 9054, 9055, 9056, 9057,
                                10465, 10466, 10467, 10468, 10469, 10470, 10471,
                                10472, 10473, 10474, 10475, 10476, 10477, 10478,
                                10479, 10480, 10481, 10482, 10483, 10484, 10485,
                                10486, 10487, 10488, 10489, 10490, 10491, 10492,
                                10493, 10494, 10495, 10496, 10497, 11910, 11911,
                                11912, 11913, 11914, 11915, 11916, 11917, 11918,
                                11919, 11920, 11921, 11922, 11923, 11924, 11925,
                                11926, 11927, 11928, 11929, 11930, 11931, 11932,
                                11933, 11934, 11935, 13345, 13346, 13347, 13348,
                                13349, 13350, 13351, 13352, 13353, 13354, 13355,
                                13356, 13357, 13358, 13359, 13360, 13361, 13362,
                                13363, 13364, 13365, 13366, 13367, 13368, 13369,
                                13370, 13371, 13372, 13373, 13374, 13375, 13376,
                                14785, 14786, 14787, 14788, 14789, 14790, 14791,
                                14792, 14793, 14794, 14795, 14796, 14797, 14798,
                                14799, 14800, 14801, 14802, 14803, 14804, 14805,
                                14806, 14807, 14808, 14809, 14810, 14811, 14812,
                                14813, 14814, 14815, 16227, 16228, 16229, 16230,
                                16231, 16232, 16233, 16234, 16235, 16236, 16237,
                                16238, 16239, 16240, 16241, 16242, 16243, 16244,
                                16245, 16246, 16247, 16248, 16249, 16250, 16251,
                                16252, 16253, 16254, 16255, 16256, 17666, 17667,
                                17668, 17669, 17670, 17671, 17672, 17673, 17674,
                                17675, 17676, 17677, 17678, 17679, 17680, 17681,
                                17682, 17683, 17684, 17685, 17686, 17687, 17688,
                                17689, 17690, 17691, 17692, 17693, 17694, 17695,
                                17696, 19105, 19106, 19107, 19108, 19109, 19110,
                                19111, 19112, 19113, 19114, 19115, 19116, 19117,
                                19118, 19119, 19120, 19121, 19122, 19123, 19124,
                                19125, 19126, 19127, 19128, 19129, 19130, 19131,
                                19132, 19133, 19134, 19135, 20545, 20546, 20547,
                                20548, 20549, 20550, 20551, 20552, 20553, 20554,
                                20555, 20556, 20557, 20558, 20559, 20560, 20561,
                                20562, 20563, 20564, 20565, 20566, 20567, 20568,
                                20569, 20570, 20571, 20572, 20573, 20574, 20575,
                                23432, 23433, 23434, 23435, 23436, 23437, 23438,
                                23439, 23440, 23441, 23442, 23443, 23444, 23445,
                                23446, 23447, 23448, 23449, 23450, 23451, 23452,
                                23453, 23454, 24868, 24869, 24870, 24871, 24872,
                                24873, 24874, 24875, 24876, 24877, 24878, 24879,
                                24880, 24881, 24882, 24883, 24884, 24885, 24886,
                                24887, 24888, 24889, 24890, 24891, 24892, 24893,
                                24894, 26311, 26312, 26313, 26316, 26317, 26318,
                                26319, 26320, 26321, 26322, 26323, 26324, 26325,
                                26326, 26327, 26328, 26329, 26330, 26331, 26332,
                                26333, 26334, 27749, 27756, 27757, 27758, 27759,
                                27760, 27761, 27762, 27763, 27766, 27767, 27768,
                                27769, 27770, 27771, 27772, 29192, 29197, 29198,
                                29199, 29200, 29201, 29202, 29203, 29204, 29205,
                                29206, 29207, 29208, 29209, 29210, 29211, 29212,
                                29213, 29214, 32072, 32073, 32074, 32075, 32076,
                                32077, 32078, 32079, 32080, 32081, 32082, 32083,
                                32084, 32085, 32086, 32087, 32088, 32089, 32090,
                                32091, 32092, 32093, 33509, 33510, 33513, 33514,
                                33515, 33516, 33517, 33518, 33519, 33520, 33521,
                                33522, 33523, 33524, 33525, 33526, 33527, 33528,
                                33529, 33530, 33531, 33532, 33533, 34948, 34949,
                                34950, 34951, 34952, 34953, 34954, 34955, 34956,
                                34957, 34958, 34959, 34960, 34961, 34962, 34963,
                                34964, 34965, 34966, 34967, 34968, 34969, 34970,
                                34971, 34972, 36388, 36389, 36390, 36391, 36392,
                                36393, 36394, 36395, 36396, 36397, 36398, 36399,
                                36400, 36401, 36402, 36403, 36404, 36405, 36406,
                                36407, 36408, 36409, 36410, 36411, 36412, 37852,
                                37853, 42152, 42153, 42154, 42155, 42156, 42157,
                                42158, 42159, 42160, 42161, 42162, 42163, 42164,
                                42165, 42166, 42167, 42168, 42169, 42170, 42171,
                                42172])

# Extract the irradiance data for the problematic indices
irradiance_values_at_indices = irradiance_array[problematic_indices]

# Print the irradiance values at the problematic indices
print("Irradiance values at problematic indices:", irradiance_values_at_indices)

# Function to extract and debug solar irradiance components at problematic indices
def debug_irradiance_at_indices(irradianceArray, indices, minutes_in_month, days_in_month, latitude, begin_month_day):
    declination = np.zeros(minutes_in_month)
    hourAngle = np.zeros(minutes_in_month)
    altitudeAngle = np.zeros(minutes_in_month)
    solarAzimuth = np.zeros(minutes_in_month)
    C = np.zeros(minutes_in_month)
    incidenceAngle = np.zeros(minutes_in_month)
    beamRad = np.zeros(minutes_in_month)
    diffuseRad = np.zeros(minutes_in_month)
    reflectedRad = np.zeros(minutes_in_month)
    collectorRad = np.zeros(minutes_in_month)

    for i in range(days_in_month):
        currentDeclination = 23.45 * math.sin(math.radians((360/365) * (begin_month_day + i)))
        minuteOfSolarNoon = solarNoon(begin_month_day + i, latitude, 150.0, 156.92, False)
        currentC = 0.095 + (0.04 * math.sin(math.radians((360/365) * (begin_month_day + i - 100))))

        for j in range(1440):  # minutes in one day = 60*24 = 1440
            currentMinute = (1440 * i) + j
            declination[currentMinute] = currentDeclination
            minutesFromSolarNoon = (minuteOfSolarNoon - j)
            hourAngle[currentMinute] = (minutesFromSolarNoon / 60.0) * 15.0
            altitudeAngle[currentMinute] = math.degrees(math.asin(math.cos(math.radians(latitude)) * 
                                        math.cos(math.radians(declination[currentMinute])) * 
                                        math.cos(math.radians(hourAngle[currentMinute])) + 
                                        (math.sin(math.radians(latitude)) * 
                                         math.sin(math.radians(declination[currentMinute])))))
            solarAzimuth[currentMinute] = math.degrees(math.asin((math.cos(math.radians(declination[currentMinute])) * 
                                      math.sin(math.radians(hourAngle[currentMinute])))/
                                      (math.cos(math.radians(altitudeAngle[currentMinute])))))
            incidenceAngle[currentMinute] = math.degrees(math.acos(math.cos(math.radians(altitudeAngle[currentMinute])) * 
                                       math.cos(math.radians(solarAzimuth[currentMinute])) * 
                                       math.sin(math.radians(latitude)) + 
                                       (math.sin(math.radians(altitudeAngle[currentMinute])) * 
                                        math.cos(math.radians(latitude)))))
            C[currentMinute] = currentC
            beamRad[currentMinute] = irradianceArray[currentMinute] * math.cos(math.radians(incidenceAngle[currentMinute]))
            diffuseRad[currentMinute] = irradianceArray[currentMinute] * currentC * ((1 + math.cos(math.radians(latitude))) / 2)
            reflectedRad[currentMinute] = irradianceArray[currentMinute] * 0.2 * (currentC + math.sin(math.radians(altitudeAngle[currentMinute]))) * ((1 - math.cos(math.radians(latitude))) / 2)
            collectorRad[currentMinute] = beamRad[currentMinute] + diffuseRad[currentMinute] + reflectedRad[currentMinute]

            if currentMinute in indices:
                print(f"Minute {currentMinute}: Irradiance = {irradianceArray[currentMinute]}, BeamRad = {beamRad[currentMinute]}, DiffuseRad = {diffuseRad[currentMinute]}, ReflectedRad = {reflectedRad[currentMinute]}, CollectorRad = {collectorRad[currentMinute]}")

    return collectorRad

# Debugging irradiance at the specified problematic indices
debug_irradiance_at_indices(irradiance_array, problematic_indices, 30*24*60, 30, 20.77,156.92,150,30,244, begin_month_day)
